In [135]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from PIL import Image
import copy #

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T


from pickle import TRUE
import torch
from torch import nn
import copy
from collections import deque
import random

import gym
from tqdm import tqdm
import pylab 
import matplotlib.pyplot as plt

In [136]:
#env = gym.make('CartPole-v0', render_mode='rgb_array').unwrapped
#env = gym.make('CartPole-v0').unwrapped

env = gym.make("CartPole-v1")
observation_space = env.observation_space.shape[0]
action_space = env.action_space.n
#DQN_Agent = DQN_Agent(observation_space, action_space)

In [137]:
def cartpole():
    env = gym.make('CartPole-v0')
    observation_space = env.observation_space.shape[0]
    action_space = env.action_space.n
    agent = DQN_Agent(observation_space, action_space)
    run = 0
    while True:
        run += 1
        state = env.reset()
        state = np.reshape(state, [1, observation_space])
        step = 0
        while True:
            step += 1
            #env.render()
            action = agent.act(state)
            state_next, reward, terminal, info = env.step(action)
            reward = reward if not terminal else -reward
            state_next = np.reshape(state_next, [1, observation_space])
            agent.remember(state, action, reward, state_next, terminal)
            state = state_next
            if terminal:
                
                break
            agent.experience_replay()

In [138]:
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display
plt.ion()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [139]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward', 'done'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))
        def remember(self, state, action, reward, next_state, done):
            self.memory.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [140]:
class DQN(nn.Module):

    def __init__(self, observation_space, action_space):
        super(DQN, self).__init__()
        

        # Number of Linear input connections depends on output of conv2d layers
        # and therefore the input image size, so compute it.
        
        self.head = nn.Linear(linear_input_size, outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = x.to(device)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return self.head(x.view(x.size(0), -1))

In [141]:
'''
resize = T.Compose([T.ToPILImage(),
                    T.Resize(40, interpolation=Image.CUBIC),
                    T.ToTensor()])


def get_cart_location(screen_width):
    world_width = env.x_threshold * 2
    scale = screen_width / world_width
    return int(env.state[0] * scale + screen_width / 2.0)  # MIDDLE OF CART

#def get_screen():
    # Returned screen requested by gym is 400x600x3, but is sometimes larger
    # such as 800x1200x3. Transpose it into torch order (CHW).
    screen = env.render().transpose((2, 0, 1))
    # Cart is in the lower half, so strip off the top and bottom of the screen
    _, screen_height, screen_width = screen.shape
    screen = screen[:, int(screen_height*0.4):int(screen_height * 0.8)]
    view_width = int(screen_width * 0.6)
    cart_location = get_cart_location(screen_width)
    if cart_location < view_width // 2:
        slice_range = slice(view_width)
    elif cart_location > (screen_width - view_width // 2):
        slice_range = slice(-view_width, None)
    else:
        slice_range = slice(cart_location - view_width // 2,
                            cart_location + view_width // 2)
    # Strip off the edges, so that we have a square image centered on a cart
    screen = screen[:, :, slice_range]
    # Convert to float, rescale, convert to torch tensor
    # (this doesn't require a copy)
    screen = np.ascontiguousarray(screen, dtype=np.float32) / 255
    screen = torch.from_numpy(screen)
    # Resize, and add a batch dimension (BCHW)
    return resize(screen).unsqueeze(0)


#env.reset()

plt.figure()
plt.imshow(get_screen().cpu().squeeze(0).permute(1, 2, 0).numpy(),
           interpolation='none')
plt.title('Example extracted screen')
plt.show()
'''

"\nresize = T.Compose([T.ToPILImage(),\n                    T.Resize(40, interpolation=Image.CUBIC),\n                    T.ToTensor()])\n\n\ndef get_cart_location(screen_width):\n    world_width = env.x_threshold * 2\n    scale = screen_width / world_width\n    return int(env.state[0] * scale + screen_width / 2.0)  # MIDDLE OF CART\n\n#def get_screen():\n    # Returned screen requested by gym is 400x600x3, but is sometimes larger\n    # such as 800x1200x3. Transpose it into torch order (CHW).\n    screen = env.render().transpose((2, 0, 1))\n    # Cart is in the lower half, so strip off the top and bottom of the screen\n    _, screen_height, screen_width = screen.shape\n    screen = screen[:, int(screen_height*0.4):int(screen_height * 0.8)]\n    view_width = int(screen_width * 0.6)\n    cart_location = get_cart_location(screen_width)\n    if cart_location < view_width // 2:\n        slice_range = slice(view_width)\n    elif cart_location > (screen_width - view_width // 2):\n        sli

In [142]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

In [143]:
#init_screen = get_screen()
#_, _, screen_height, screen_width = init_screen.shape
n_actions = env.action_space.n
#policy_net = DQN(screen_height, screen_width, n_actions).to(device)
#target_net = DQN(screen_height, screen_width, n_actions).to(device)

#target_net.load_state_dict(policy_net.state_dict())
#target_net.eval()

##self.target_net = self.build_nn(layer_sizes)
#self.policy_net = copy.deepcopy(self.target_net)

#        self.target_net = self.build_nn(layer_sizes)
#        self.policy_net = self.target_net
#        self.target_net.cpu()
#        self.policy_net.cpu()


In [144]:
memory = ReplayMemory(10000)
steps_done = 0

class DQN_Agent:

    def __init__(self, seed, layer_sizes, lr, sync_freq, exp_replay_size):
        torch.manual_seed(seed)

        self.target_net = self.build_nn(layer_sizes)
        self.policy_net = copy.deepcopy(self.target_net)

        self.target_net = self.build_nn(layer_sizes)
        self.policy_net = self.target_net
        self.target_net.cpu()
        self.policy_net.cpu()
        self.loss_fn = torch.nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.target_net.parameters(), lr=lr)
        #self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))
        #optimizer = optim.RMSprop(policy_net.parameters()) # want to play with optim.Adam #<----------------
        #optimizer = torch.optim.Adam(self.target_net.parameters(), lr=lr)
        

        
        

        self.network_sync_freq = sync_freq
        self.network_sync_counter = 0
        self.gamma = torch.tensor(0.95).float().cpu()
        self.experience_replay = deque(maxlen=exp_replay_size) #deque = double-ended queue adds and removes items from both ends
        return

    def build_nn(self, layer_sizes):
        assert len(layer_sizes) > 1
        layers = []
        for index in range(len(layer_sizes) - 1):
            linear = nn.Linear(layer_sizes[index], layer_sizes[index + 1])
            act = nn.Tanh() if index < len(layer_sizes) - 2 else nn.Identity()
            layers += (linear, act)
        return nn.Sequential(*layers)
    


    def select_action(self, state, action_space_len):
        global steps_done
        sample = random.random()
        eps_threshold = EPS_END + (EPS_START - EPS_END) * \
            math.exp(-1. * steps_done / EPS_DECAY)
        steps_done += 1
        if sample > eps_threshold:
            with torch.no_grad():
                # t.max(1) will return largest column value of each row.
                # second column on max result is index of where max element was
                # found, so we pick action with the larger expected reward.
                return policy_net(state).max(1)[1].view(1, 1)



        else:
            return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)



In [145]:
episode_durations = []

def plot_durations():
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())

In [146]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch) # torch.gather()

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [147]:
num_episodes = 50
for i_episode in range(num_episodes):
    # Initialize the environment and state
    env.reset()

    state, action, next_state, reward = env.reset()[0] ## ERROR HERE
    obs = env.reset()[0]
    
    input_dim = env.observation_space.shape[0]
    output_dim = env.action_space.n
    exp_replay_size = 256
    agent = DQN_Agent(seed=1423, layer_sizes=[input_dim, 64, output_dim], lr=1e-3, sync_freq=5,
                    exp_replay_size=exp_replay_size)

  
    
    #last_screen = get_screen()
    #current_screen = get_screen()
    
    for t in count():
        # Select and perform an action
        action = agent.select_action(obs, state)
        state, reward, done, _, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)

        # Observe new state
        #state = next_state
        #if not done:
        #    state = next_state
        #else:
        #    state = None

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()
        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')
env.render()
env.close()
plt.ioff()
plt.show()

NameError: name 'steps_done' is not defined